In [ ]:
# Simple Flask Server to Sync a Web Textbox to the Local Linux Clipboard
#
# Description:
# This script starts a web server on localhost:5000. When you open the page
# in a browser, it displays a large textbox. Any text you type into this
# box is automatically sent to the server and copied to the system's
# primary clipboard using the 'xclip' utility.
#
# Prerequisites:
# 1. Flask: You must have Flask installed.
#    pip install Flask
#
# 2. xclip: This utility must be installed on your Linux machine.
#    On Debian/Ubuntu: sudo apt-get install xclip
#    On Fedora/CentOS: sudo dnf install xclip
#
# How to Run:
# 1. Save this code as a Python file (e.g., clipboard_server.py).
# 2. Run it from your terminal: python clipboard_server.py
# 3. Open your web browser and navigate to http://127.0.0.1:5000

import subprocess
import sys
from flask import Flask, render_template_string, request, jsonify

def check_for_xclip():
    """Checks if the 'xclip' command is available on the system."""
    try:
        # Use subprocess.run to check for the command's existence.
        # stdout and stderr are redirected to DEVNULL to keep the console clean.
        subprocess.run(['which', 'xclip'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

# --- Flask App Initialization ---
app = Flask(__name__)

# --- HTML, CSS, and JavaScript Template ---
# This single string contains all the frontend code for our application.
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Local Clipboard Sync</title>
    <style>
        body {
            font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif;
            background-color: #1a1a1a;
            color: #e0e0e0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            padding: 20px;
            box-sizing: border-box;
        }
        .container {
            width: 100%;
            max-width: 800px;
            text-align: center;
        }
        h1 {
            color: #ffffff;
            font-weight: 500;
        }
        p {
            color: #999999;
            margin-bottom: 20px;
        }
        textarea {
            width: 100%;
            height: 60vh;
            background-color: #2b2b2b;
            color: #e0e0e0;
            border: 1px solid #444;
            border-radius: 8px;
            padding: 15px;
            font-size: 16px;
            box-sizing: border-box;
            resize: none;
            transition: border-color 0.3s, box-shadow 0.3s;
        }
        textarea:focus {
            outline: none;
            border-color: #007bff;
            box-shadow: 0 0 8px rgba(0, 123, 255, 0.6);
        }
        #status {
            margin-top: 15px;
            height: 20px;
            color: #28a745;
            font-weight: 500;
            transition: opacity 0.5s ease-out;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Real-time Clipboard Sync</h1>
        <p>Text you type here is copied to the clipboard of the machine running this server.</p>
        <textarea id="clipboard-text" placeholder="Start typing..."></textarea>
        <p id="status"></p>
    </div>

    <script>
        const textarea = document.getElementById('clipboard-text');
        const statusElement = document.getElementById('status');
        let statusTimeout;

        // This function sends the textarea content to the Flask backend via a POST request.
        const syncToClipboard = async () => {
            const text = textarea.value;
            try {
                const response = await fetch('/copy', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ text: text }),
                });

                if (response.ok) {
                    // Update status message on successful copy.
                    statusElement.textContent = 'Copied to clipboard!';
                    statusElement.style.opacity = '1';

                    // Clear any existing timeout to reset the fade-out timer.
                    clearTimeout(statusTimeout);

                    // Set a new timeout to make the message disappear after a moment.
                    statusTimeout = setTimeout(() => {
                        statusElement.style.opacity = '0';
                    }, 1500);
                } else {
                    statusElement.textContent = 'Error: Could not copy.';
                    statusElement.style.opacity = '1';
                }
            } catch (error) {
                console.error('Fetch error:', error);
                statusElement.textContent = 'Error: Server is unreachable.';
                statusElement.style.opacity = '1';
            }
        };

        // Add an 'input' event listener to the textarea to trigger the sync on every keystroke.
        textarea.addEventListener('input', syncToClipboard);
    </script>
</body>
</html>
"""

# --- Flask Routes ---

@app.route('/')
def index():
    """Serves the main HTML page."""
    return render_template_string(HTML_TEMPLATE)

@app.route('/copy', methods=['POST'])
def copy_to_clipboard():
    """
    Receives text from the frontend and copies it to the system clipboard using xclip.
    This is an API endpoint that only accepts POST requests.
    """
    try:
        data = request.get_json()
        if data is None or 'text' not in data:
            return jsonify({'status': 'error', 'message': 'Invalid JSON or missing text key'}), 400

        text_to_copy = data['text']

        # Use subprocess.run to pipe the text to xclip. This is the core command
        # that interacts with the Linux clipboard.
        subprocess.run(
            ['xclip', '-selection', 'clipboard'],
            input=text_to_copy,
            text=True,
            check=True
        )
        return jsonify({'status': 'success'})
    except FileNotFoundError:
        # This case is unlikely if the startup check passes, but it's good practice.
        return jsonify({'status': 'error', 'message': 'xclip command not found on server'}), 500
    except subprocess.CalledProcessError as e:
        # Handle cases where xclip fails for some reason.
        return jsonify({'status': 'error', 'message': f'xclip failed with error: {e}'}), 500
    except Exception as e:
        # Catch any other unexpected errors.
        return jsonify({'status': 'error', 'message': str(e)}), 500

# --- Main Execution Block ---
if __name__ == '__main__':
    # It's crucial to check for dependencies before starting the server.
    if not check_for_xclip():
        print("Error: 'xclip' is not installed or not in your PATH.", file=sys.stderr)
        print("Please install it to use this application (e.g., 'sudo apt-get install xclip').", file=sys.stderr)
        sys.exit(1)

    print("Clipboard server is starting...")
    print("Navigate to http://127.0.0.1:5000 in your web browser.")
    # Run the Flask app. We set debug=False as it's not needed for this utility.
    app.run(host='127.0.0.1', port=5000, debug=False)